In [1]:
import tkinter as tk
from datetime import datetime
from tkinter import filedialog as fd
import openpyxl as opx
import pandas as pd
import numpy as np
import datetime as dt
from dateutil import parser as dp
from dateutil import relativedelta as rd

# Status Flags
cintGreater24 = 16
cintNotDate = 32
cintEndDateAssumed = 8
cintStartDateAssumed = 4
cintDatesSwapped = 2
cintEndAdjusted = 1

pd.options.display.max_rows = 50
pd.set_option('display.precision', 2)
pd.set_option('display.float_format', lambda x: f'{x:.2f}')

def generate_hour_labels_and_dict():
    hour_labels = [f"{hour:02d}:00-{hour:02d}:59" for hour in range(24)]
    hour_dict = {hour: label for hour, label in enumerate(hour_labels)}
    return hour_labels, hour_dict

# Generate hour labels and dictionary
hour_labels, hour_dict = generate_hour_labels_and_dict()

# Creating a DataFrame with hour labels as column headings
dfHours = pd.DataFrame(columns=hour_labels)


def is_date(dtcheck) -> bool:
    try:
        #print(dtcheck)
        result = bool((dp.parse(str(dtcheck))))
    except:
        result = False
    # print(result)
    return result


def hour_span(dfrom, dto):
    return abs((dto - dfrom).total_seconds() / 3600)


def clean_date(dfdates) -> list:
    #
    result = list()
    from_good = is_date(dfdates.loc['From'])
    to_good = is_date(dfdates.loc['To'])
    dfrom = dt.datetime.now()
    dto = dt.datetime.now()
    cross_mid = False  # span crosses midnight
    status = 0  # transform status
    span = 0  # hours in span
    # do we have at least 1 date
    if from_good or to_good:
        # only one date is good use existing in empty date
        if not from_good:
            dfrom: datetime = dfdates.loc['To']
            dto: datetime = dfdates.loc['To']
            status += cintStartDateAssumed
            result.append(dto.date())            
            result.append(dfrom.date())
            result.append(dfrom.hour + (dfrom.minute/60))
            result.append(dfrom.hour)
            result.append(dto.hour + (dto.minute/60))
            result.append(dto.hour)
            result.append(span)
            result.append(cross_mid)
            result.append(dto)
            result.append(dfrom)
            result.append(status)
        elif not to_good:
            dfrom: datetime = dfdates.loc['From']
            dto: datetime = dfdates.loc['From']
            status += cintEndDateAssumed
            result.append(dto.date())            
            result.append(dfrom.date())
            result.append(dfrom.hour + (dfrom.minute/60))
            result.append(dfrom.hour)
            result.append(dto.hour + (dto.minute/60))
            result.append(dto.hour)
            result.append(span)
            result.append(cross_mid)
            result.append(dto)
            result.append(dfrom)
            result.append(status)
        # do we have two dates
        else:
            # We have two dates is the span greater than 24 hours
            dfrom: datetime = dfdates.loc['From']
            dto = dfdates.loc['To']
            span = hour_span(dfrom,dto)
            if span > 25:
                # span > 24 hours return status code
                result.append(dto.date())            
                result.append(dfrom.date())
                result.append(dfrom.hour + (dfrom.minute/60))
                result.append(dfrom.hour)
                result.append(dto.hour + (dto.minute/60))
                result.append(dto.hour)
                result.append(span)
                result.append(cross_mid)
                result.append(dto)
                result.append(dfrom)
                result.append(cintGreater24)
            else:
                # span is good check that dates are in right order
                if (dfrom > dto) and (dfrom != dto):
                    # Dates are in wrong order swap them
                    dfrom, dto = dto, dfrom
                    status += cintDatesSwapped
                    # Check the to time back off 1 minute if on the hour

                if dfrom != dto and dto != dto.minute == 0:
                    dto = dto - rd.relativedelta(minutes=1)
                    status += cintEndAdjusted
                # check if time crosses midnight
                if (dto.date() - dfrom.date()).days > 0:
                    cross_mid = True
                result.append(dto.date())            
                result.append(dfrom.date())
                result.append(dfrom.hour + (dfrom.minute/60))
                result.append(dfrom.hour)
                result.append(dto.hour + (dto.minute/60))
                result.append(dto.hour)
                result.append(span)
                result.append(cross_mid)
                result.append(dto)
                result.append(dfrom)
                result.append(status)

    else:
        # we have no dates return status code
        result.append(np.NAN)
        result.append(np.NAN)
        result.append(np.NAN)
        result.append(np.NAN)
        result.append(np.NAN)
        result.append(np.NAN)
        result.append(np.NAN)
        result.append(cross_mid)
        result.append(np.NAN)
        result.append(np.NAN)
        result.append(cintNotDate)

    return result

def parse_status(status_code):
    status_mapping = {0: 'None', 1: 'End Time Adjusted', 2: 'Dates Swapped', 3: 'Dates Swapped and End Time Adjusted',
                      4: 'End Date Used For Start Date', 8: 'Start Date Used For End Date', 16: 'Longer Than 24 HR',
                      32: 'No Dates Supplied', 64: 'Duplicate'}
    return status_mapping.get(status_code, 'unknown')
        


filename = fd.askopenfilename(title='Select file', filetypes=[('Excel Files', ['*.xlsx', '*.xlsm', '*.xls', '*.csv'])])

if filename.endswith('.csv'):
    wb_in = pd.read_csv(filename)
else:
    wb_in = pd.read_excel(filename)

#
wb_in = wb_in.assign(CleanFrom = np.NAN, CleanTo = np.NAN, DPFrom = np.NAN, DPTo = np.NAN, FromDec = np.NAN,
                      ToDec = np.NAN, FromHour = np.NAN, ToHour = np.NAN,
                     DBH = 0, CrossMid = 0, Span = 0, Status = 0)

wb_in  


,OCC,From,To,CleanFrom,CleanTo,DPFrom,DPTo,FromDec,ToDec,FromHour,ToHour,DBH,CrossMid,Span,Status
0,1615747,2006-06-19 23:00:00,2006-06-20 01:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
1,1436665,2006-07-02 01:45:00,2006-07-02 02:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
2,1644913,2006-07-04 00:05:00,2006-07-04 01:15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
3,1633589,2006-07-06 21:30:00,2006-07-06 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
4,1674345,2006-07-10 00:30:00,2006-07-10 00:37:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
5,1674349,2006-07-10 01:20:00,2006-07-10 01:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
6,1514945,2006-07-13 13:30:00,2006-07-13 13:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
7,1710300,2006-07-14 13:05:00,2006-07-14 13:18:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
8,1712973,2006-07-19 01:30:00,2006-07-19 01:39:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
9,1712973,2006-07-19 01:30:00,2006-07-19 01:39:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0


In [2]:
# Clean and parse the date data
# look at each row and check to see if there is at least 1 date
for r in wb_in.index:
    clean_result = clean_date(wb_in.loc[r, ['From', 'To']])
    wb_in.at[r, 'Status'] = clean_result.pop()
    wb_in.at[r, 'CleanFrom'] = clean_result.pop()
    wb_in.at[r, 'CleanTo'] = clean_result.pop()
    wb_in.at[r, 'CrossMid'] = clean_result.pop()
    wb_in.at[r, 'Span'] = clean_result.pop()
    wb_in.at[r,'ToHour'] = clean_result.pop()
    wb_in.at[r,'ToDec'] = clean_result.pop()
    wb_in.at[r,'FromHour'] = clean_result.pop()
    wb_in.at[r,'FromDec'] = clean_result.pop()
    wb_in.at[r,'DPFrom'] = clean_result.pop()
    wb_in.at[r,'DPTo'] = clean_result.pop()

wb_in

,OCC,From,To,CleanFrom,CleanTo,DPFrom,DPTo,FromDec,ToDec,FromHour,ToHour,DBH,CrossMid,Span,Status
0,1615747,2006-06-19 23:00:00,2006-06-20 01:30:00,2006-06-19 23:00:00,2006-06-20 01:30:00,2006-06-19,2006-06-20,23.00,1.50,23.00,1.00,0,True,2.50,0
1,1436665,2006-07-02 01:45:00,2006-07-02 02:10:00,2006-07-02 01:45:00,2006-07-02 02:10:00,2006-07-02,2006-07-02,1.75,2.17,1.00,2.00,0,False,0.42,0
2,1644913,2006-07-04 00:05:00,2006-07-04 01:15:00,2006-07-04 00:05:00,2006-07-04 01:15:00,2006-07-04,2006-07-04,0.08,1.25,0.00,1.00,0,False,1.17,0
3,1633589,2006-07-06 21:30:00,2006-07-06 23:00:00,2006-07-06 21:30:00,2006-07-06 22:59:00,2006-07-06,2006-07-06,21.50,22.98,21.00,22.00,0,False,1.50,1
4,1674345,2006-07-10 00:30:00,2006-07-10 00:37:00,2006-07-10 00:30:00,2006-07-10 00:37:00,2006-07-10,2006-07-10,0.50,0.62,0.00,0.00,0,False,0.12,0
5,1674349,2006-07-10 01:20:00,2006-07-10 01:30:00,2006-07-10 01:20:00,2006-07-10 01:30:00,2006-07-10,2006-07-10,1.33,1.50,1.00,1.00,0,False,0.17,0
6,1514945,2006-07-13 13:30:00,2006-07-13 13:35:00,2006-07-13 13:30:00,2006-07-13 13:35:00,2006-07-13,2006-07-13,13.50,13.58,13.00,13.00,0,False,0.08,0
7,1710300,2006-07-14 13:05:00,2006-07-14 13:18:00,2006-07-14 13:05:00,2006-07-14 13:18:00,2006-07-14,2006-07-14,13.08,13.30,13.00,13.00,0,False,0.22,0
8,1712973,2006-07-19 01:30:00,2006-07-19 01:39:00,2006-07-19 01:30:00,2006-07-19 01:39:00,2006-07-19,2006-07-19,1.50,1.65,1.00,1.00,0,False,0.15,0
9,1712973,2006-07-19 01:30:00,2006-07-19 01:39:00,2006-07-19 01:30:00,2006-07-19 01:39:00,2006-07-19,2006-07-19,1.50,1.65,1.00,1.00,0,False,0.15,0


In [3]:
wb_nodup = wb_in.drop_duplicates(keep='first')
wb_excluded = wb_in[wb_in.duplicated(keep='first')]
wb_excluded.loc[:,'Status'] = 64
wb_excluded

,OCC,From,To,CleanFrom,CleanTo,DPFrom,DPTo,FromDec,ToDec,FromHour,ToHour,DBH,CrossMid,Span,Status
9,1712973,2006-07-19 01:30:00,2006-07-19 01:39:00,2006-07-19 01:30:00,2006-07-19 01:39:00,2006-07-19,2006-07-19,1.50,1.65,1.00,1.00,0,False,0.15,64


In [4]:
wb_nodup

,OCC,From,To,CleanFrom,CleanTo,DPFrom,DPTo,FromDec,ToDec,FromHour,ToHour,DBH,CrossMid,Span,Status
0,1615747,2006-06-19 23:00:00,2006-06-20 01:30:00,2006-06-19 23:00:00,2006-06-20 01:30:00,2006-06-19,2006-06-20,23.00,1.50,23.00,1.00,0,True,2.50,0
1,1436665,2006-07-02 01:45:00,2006-07-02 02:10:00,2006-07-02 01:45:00,2006-07-02 02:10:00,2006-07-02,2006-07-02,1.75,2.17,1.00,2.00,0,False,0.42,0
2,1644913,2006-07-04 00:05:00,2006-07-04 01:15:00,2006-07-04 00:05:00,2006-07-04 01:15:00,2006-07-04,2006-07-04,0.08,1.25,0.00,1.00,0,False,1.17,0
3,1633589,2006-07-06 21:30:00,2006-07-06 23:00:00,2006-07-06 21:30:00,2006-07-06 22:59:00,2006-07-06,2006-07-06,21.50,22.98,21.00,22.00,0,False,1.50,1
4,1674345,2006-07-10 00:30:00,2006-07-10 00:37:00,2006-07-10 00:30:00,2006-07-10 00:37:00,2006-07-10,2006-07-10,0.50,0.62,0.00,0.00,0,False,0.12,0
5,1674349,2006-07-10 01:20:00,2006-07-10 01:30:00,2006-07-10 01:20:00,2006-07-10 01:30:00,2006-07-10,2006-07-10,1.33,1.50,1.00,1.00,0,False,0.17,0
6,1514945,2006-07-13 13:30:00,2006-07-13 13:35:00,2006-07-13 13:30:00,2006-07-13 13:35:00,2006-07-13,2006-07-13,13.50,13.58,13.00,13.00,0,False,0.08,0
7,1710300,2006-07-14 13:05:00,2006-07-14 13:18:00,2006-07-14 13:05:00,2006-07-14 13:18:00,2006-07-14,2006-07-14,13.08,13.30,13.00,13.00,0,False,0.22,0
8,1712973,2006-07-19 01:30:00,2006-07-19 01:39:00,2006-07-19 01:30:00,2006-07-19 01:39:00,2006-07-19,2006-07-19,1.50,1.65,1.00,1.00,0,False,0.15,0
10,1684850,2006-07-22 23:50:00,2006-07-22 23:55:00,2006-07-22 23:50:00,2006-07-22 23:55:00,2006-07-22,2006-07-22,23.83,23.92,23.00,23.00,0,False,0.08,0


In [6]:
wb_excluded = pd.concat([wb_excluded, wb_nodup[wb_nodup['Status']>
                                               15]])
wb_excluded                    

,OCC,From,To,CleanFrom,CleanTo,DPFrom,DPTo,FromDec,ToDec,FromHour,ToHour,DBH,CrossMid,Span,Status
9,1712973,2006-07-19 01:30:00,2006-07-19 01:39:00,2006-07-19 01:30:00,2006-07-19 01:39:00,2006-07-19,2006-07-19,1.50,1.65,1.00,1.00,0,False,0.15,64
26,9546778,y,y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,False,NaN,32
28,9344561,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,False,NaN,32
29,9866552,2006-06-19 23:00:00,2006-08-03 22:20:00,2006-06-19 23:00:00,2006-08-03 22:20:00,2006-06-19,2006-08-03,23.00,22.33,23.00,22.00,0,False,1079.33,16


In [7]:
wb_excluded = wb_excluded[['OCC','From','To', 'Status']]
wb_excluded

,OCC,From,To,Status
9,1712973,2006-07-19 01:30:00,2006-07-19 01:39:00,64
26,9546778,y,y,32
28,9344561,NaN,NaN,32
29,9866552,2006-06-19 23:00:00,2006-08-03 22:20:00,16


In [10]:
wb_excluded['Reason'] = wb_excluded['Status'].apply(parse_status)

/var/folders/c9/w_lrh4ls5zxcjt8rkk39qx9r0000gn/T/ipykernel_4272/2997251445.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wb_excluded['Reason'] = wb_excluded.loc[:,'Status'].apply(parse_status)


In [13]:
wb_nodup['Adjustment'] = wb_nodup.loc[:,'Status'].apply(parse_status)

/var/folders/c9/w_lrh4ls5zxcjt8rkk39qx9r0000gn/T/ipykernel_4272/4034291404.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wb_nodup['Adjustment'] = wb_nodup.loc[:,'Status'].apply(parse_status)


In [14]:
wb_nodup

,OCC,From,To,CleanFrom,CleanTo,DPFrom,DPTo,FromDec,ToDec,FromHour,ToHour,DBH,CrossMid,Span,Status,Adjustment
0,1615747,2006-06-19 23:00:00,2006-06-20 01:30:00,2006-06-19 23:00:00,2006-06-20 01:30:00,2006-06-19,2006-06-20,23.00,1.50,23.00,1.00,0,True,2.50,0,None
1,1436665,2006-07-02 01:45:00,2006-07-02 02:10:00,2006-07-02 01:45:00,2006-07-02 02:10:00,2006-07-02,2006-07-02,1.75,2.17,1.00,2.00,0,False,0.42,0,None
2,1644913,2006-07-04 00:05:00,2006-07-04 01:15:00,2006-07-04 00:05:00,2006-07-04 01:15:00,2006-07-04,2006-07-04,0.08,1.25,0.00,1.00,0,False,1.17,0,None
3,1633589,2006-07-06 21:30:00,2006-07-06 23:00:00,2006-07-06 21:30:00,2006-07-06 22:59:00,2006-07-06,2006-07-06,21.50,22.98,21.00,22.00,0,False,1.50,1,End Time Adjusted
4,1674345,2006-07-10 00:30:00,2006-07-10 00:37:00,2006-07-10 00:30:00,2006-07-10 00:37:00,2006-07-10,2006-07-10,0.50,0.62,0.00,0.00,0,False,0.12,0,None
5,1674349,2006-07-10 01:20:00,2006-07-10 01:30:00,2006-07-10 01:20:00,2006-07-10 01:30:00,2006-07-10,2006-07-10,1.33,1.50,1.00,1.00,0,False,0.17,0,None
6,1514945,2006-07-13 13:30:00,2006-07-13 13:35:00,2006-07-13 13:30:00,2006-07-13 13:35:00,2006-07-13,2006-07-13,13.50,13.58,13.00,13.00,0,False,0.08,0,None
7,1710300,2006-07-14 13:05:00,2006-07-14 13:18:00,2006-07-14 13:05:00,2006-07-14 13:18:00,2006-07-14,2006-07-14,13.08,13.30,13.00,13.00,0,False,0.22,0,None
8,1712973,2006-07-19 01:30:00,2006-07-19 01:39:00,2006-07-19 01:30:00,2006-07-19 01:39:00,2006-07-19,2006-07-19,1.50,1.65,1.00,1.00,0,False,0.15,0,None
10,1684850,2006-07-22 23:50:00,2006-07-22 23:55:00,2006-07-22 23:50:00,2006-07-22 23:55:00,2006-07-22,2006-07-22,23.83,23.92,23.00,23.00,0,False,0.08,0,None


In [17]:
wb_final = wb_nodup[wb_nodup['Status']<= 15]

In [18]:
wb_final

,OCC,From,To,CleanFrom,CleanTo,DPFrom,DPTo,FromDec,ToDec,FromHour,ToHour,DBH,CrossMid,Span,Status,Adjustment
0,1615747,2006-06-19 23:00:00,2006-06-20 01:30:00,2006-06-19 23:00:00,2006-06-20 01:30:00,2006-06-19,2006-06-20,23.00,1.50,23.00,1.00,0,True,2.50,0,None
1,1436665,2006-07-02 01:45:00,2006-07-02 02:10:00,2006-07-02 01:45:00,2006-07-02 02:10:00,2006-07-02,2006-07-02,1.75,2.17,1.00,2.00,0,False,0.42,0,None
2,1644913,2006-07-04 00:05:00,2006-07-04 01:15:00,2006-07-04 00:05:00,2006-07-04 01:15:00,2006-07-04,2006-07-04,0.08,1.25,0.00,1.00,0,False,1.17,0,None
3,1633589,2006-07-06 21:30:00,2006-07-06 23:00:00,2006-07-06 21:30:00,2006-07-06 22:59:00,2006-07-06,2006-07-06,21.50,22.98,21.00,22.00,0,False,1.50,1,End Time Adjusted
4,1674345,2006-07-10 00:30:00,2006-07-10 00:37:00,2006-07-10 00:30:00,2006-07-10 00:37:00,2006-07-10,2006-07-10,0.50,0.62,0.00,0.00,0,False,0.12,0,None
5,1674349,2006-07-10 01:20:00,2006-07-10 01:30:00,2006-07-10 01:20:00,2006-07-10 01:30:00,2006-07-10,2006-07-10,1.33,1.50,1.00,1.00,0,False,0.17,0,None
6,1514945,2006-07-13 13:30:00,2006-07-13 13:35:00,2006-07-13 13:30:00,2006-07-13 13:35:00,2006-07-13,2006-07-13,13.50,13.58,13.00,13.00,0,False,0.08,0,None
7,1710300,2006-07-14 13:05:00,2006-07-14 13:18:00,2006-07-14 13:05:00,2006-07-14 13:18:00,2006-07-14,2006-07-14,13.08,13.30,13.00,13.00,0,False,0.22,0,None
8,1712973,2006-07-19 01:30:00,2006-07-19 01:39:00,2006-07-19 01:30:00,2006-07-19 01:39:00,2006-07-19,2006-07-19,1.50,1.65,1.00,1.00,0,False,0.15,0,None
10,1684850,2006-07-22 23:50:00,2006-07-22 23:55:00,2006-07-22 23:50:00,2006-07-22 23:55:00,2006-07-22,2006-07-22,23.83,23.92,23.00,23.00,0,False,0.08,0,None


In [19]:
wb_final = wb_final.sort_values(by=['CleanFrom', 'CleanTo'])
wb_final

,OCC,From,To,CleanFrom,CleanTo,DPFrom,DPTo,FromDec,ToDec,FromHour,ToHour,DBH,CrossMid,Span,Status,Adjustment
0,1615747,2006-06-19 23:00:00,2006-06-20 01:30:00,2006-06-19 23:00:00,2006-06-20 01:30:00,2006-06-19,2006-06-20,23.00,1.50,23.00,1.00,0,True,2.50,0,None
1,1436665,2006-07-02 01:45:00,2006-07-02 02:10:00,2006-07-02 01:45:00,2006-07-02 02:10:00,2006-07-02,2006-07-02,1.75,2.17,1.00,2.00,0,False,0.42,0,None
2,1644913,2006-07-04 00:05:00,2006-07-04 01:15:00,2006-07-04 00:05:00,2006-07-04 01:15:00,2006-07-04,2006-07-04,0.08,1.25,0.00,1.00,0,False,1.17,0,None
3,1633589,2006-07-06 21:30:00,2006-07-06 23:00:00,2006-07-06 21:30:00,2006-07-06 22:59:00,2006-07-06,2006-07-06,21.50,22.98,21.00,22.00,0,False,1.50,1,End Time Adjusted
4,1674345,2006-07-10 00:30:00,2006-07-10 00:37:00,2006-07-10 00:30:00,2006-07-10 00:37:00,2006-07-10,2006-07-10,0.50,0.62,0.00,0.00,0,False,0.12,0,None
5,1674349,2006-07-10 01:20:00,2006-07-10 01:30:00,2006-07-10 01:20:00,2006-07-10 01:30:00,2006-07-10,2006-07-10,1.33,1.50,1.00,1.00,0,False,0.17,0,None
6,1514945,2006-07-13 13:30:00,2006-07-13 13:35:00,2006-07-13 13:30:00,2006-07-13 13:35:00,2006-07-13,2006-07-13,13.50,13.58,13.00,13.00,0,False,0.08,0,None
7,1710300,2006-07-14 13:05:00,2006-07-14 13:18:00,2006-07-14 13:05:00,2006-07-14 13:18:00,2006-07-14,2006-07-14,13.08,13.30,13.00,13.00,0,False,0.22,0,None
8,1712973,2006-07-19 01:30:00,2006-07-19 01:39:00,2006-07-19 01:30:00,2006-07-19 01:39:00,2006-07-19,2006-07-19,1.50,1.65,1.00,1.00,0,False,0.15,0,None
10,1684850,2006-07-22 23:50:00,2006-07-22 23:55:00,2006-07-22 23:50:00,2006-07-22 23:55:00,2006-07-22,2006-07-22,23.83,23.92,23.00,23.00,0,False,0.08,0,None


In [20]:
wb_final.reset_index(inplace=True, drop=True)
wb_final

,OCC,From,To,CleanFrom,CleanTo,DPFrom,DPTo,FromDec,ToDec,FromHour,ToHour,DBH,CrossMid,Span,Status,Adjustment
0,1615747,2006-06-19 23:00:00,2006-06-20 01:30:00,2006-06-19 23:00:00,2006-06-20 01:30:00,2006-06-19,2006-06-20,23.00,1.50,23.00,1.00,0,True,2.50,0,None
1,1436665,2006-07-02 01:45:00,2006-07-02 02:10:00,2006-07-02 01:45:00,2006-07-02 02:10:00,2006-07-02,2006-07-02,1.75,2.17,1.00,2.00,0,False,0.42,0,None
2,1644913,2006-07-04 00:05:00,2006-07-04 01:15:00,2006-07-04 00:05:00,2006-07-04 01:15:00,2006-07-04,2006-07-04,0.08,1.25,0.00,1.00,0,False,1.17,0,None
3,1633589,2006-07-06 21:30:00,2006-07-06 23:00:00,2006-07-06 21:30:00,2006-07-06 22:59:00,2006-07-06,2006-07-06,21.50,22.98,21.00,22.00,0,False,1.50,1,End Time Adjusted
4,1674345,2006-07-10 00:30:00,2006-07-10 00:37:00,2006-07-10 00:30:00,2006-07-10 00:37:00,2006-07-10,2006-07-10,0.50,0.62,0.00,0.00,0,False,0.12,0,None
5,1674349,2006-07-10 01:20:00,2006-07-10 01:30:00,2006-07-10 01:20:00,2006-07-10 01:30:00,2006-07-10,2006-07-10,1.33,1.50,1.00,1.00,0,False,0.17,0,None
6,1514945,2006-07-13 13:30:00,2006-07-13 13:35:00,2006-07-13 13:30:00,2006-07-13 13:35:00,2006-07-13,2006-07-13,13.50,13.58,13.00,13.00,0,False,0.08,0,None
7,1710300,2006-07-14 13:05:00,2006-07-14 13:18:00,2006-07-14 13:05:00,2006-07-14 13:18:00,2006-07-14,2006-07-14,13.08,13.30,13.00,13.00,0,False,0.22,0,None
8,1712973,2006-07-19 01:30:00,2006-07-19 01:39:00,2006-07-19 01:30:00,2006-07-19 01:39:00,2006-07-19,2006-07-19,1.50,1.65,1.00,1.00,0,False,0.15,0,None
9,1684850,2006-07-22 23:50:00,2006-07-22 23:55:00,2006-07-22 23:50:00,2006-07-22 23:55:00,2006-07-22,2006-07-22,23.83,23.92,23.00,23.00,0,False,0.08,0,None
